### **CH 9 NEURAL NETWORKS**

## No Code yet?

In [ ]:
#from google.colab import drive
#drive.mount("/content/gdrive")

from google.colab import drive
drive.mount('/gdrive')
folder = "/gdrive/My Drive/Python Practice/Datasets"

### **HANDS-ON ANALYSIS**


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
np.random.seed(3)
tf.random.set_seed(3)

### 17
* a. Create binary variable that equals 1 if 'Cap_Gain_Losses' > 0 as 'CapGainsLossesPositive'

In [ ]:
adult_train = pd.read_csv(folder + '/adult_ch6_training', delimiter=',')
adult_test = pd.read_csv(folder + '/adult_ch6_test', delimiter=',')

In [ ]:
import statsmodels.tools.tools as stattools

# change CapGainsLosses to binary
adult_train["CapGainsLossesPositive"] = list(map(int, adult_train["Cap_Gains_Losses"]>0))

# change Income to numeric(binary)
adult_train["Income_factor"] = list(map(int, adult_train["Income"] == '>50K'))
#adult_train["Marital_factor"] = list(map(int, adult_train["Marital status"] == 'Married')) ####
#X_adult=adult_train[['CapGainsLossesPositive','Marital_factor']]

# convert 'Marital status' to factor
mar_np = np.array(adult_train['Marital status'])
(mar_cat, mar_cat_dict) = stattools.categorical(mar_np, drop=True, dictnames=True)

In [ ]:
mar_cat_pd = pd.DataFrame(mar_cat)
X_adult = pd.concat((adult_train['CapGainsLossesPositive'], mar_cat_pd), axis=1)

X_adult_names = ['CapGainsLossesPositive', 'Divorced', 'Married', 'Never-married', 'Separated', 'Widowed']
Y_adult_names = ['<=50K', '>50K']

* b. Convert 'Marital.status', 'Income', 'CapGainsLossesPositive' to factors

In [ ]:
X_adult.head()

### 18
Using training set, Create Neural Network model (NNM1 : model 1) to predict customer's Income

In [ ]:
#X_adult.head()
y_adult = adult_train['Income_factor']

In [ ]:
model1 = Sequential()
model1.add(Dense(10, input_dim = 6, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model1.fit(X_adult,y_adult, epochs=5, batch_size=1)

In [ ]:
print('\n Accuracy: %.4f' % (model1.evaluate(X_adult, y_adult)[1]))

In [ ]:
adult_test["CapGainsLossesPositive"] = list(map(int, adult_test["Cap_Gains_Losses"]>0))
adult_test["Income_factor"] = list(map(int, adult_test["Income"] == '>50K'))
#adult_test["Marital_factor"] = list(map(int, adult_test["Marital status"] == 'Married'))

mar_np_test = np.array(adult_test['Marital status'])
(mar_cat_test, mar_cat_dict_test) = stattools.categorical(mar_np_test, drop=True, dictnames=True)

In [ ]:
mar_cat_test_pd = pd.DataFrame(mar_cat_test)
X_adult_test = pd.concat((adult_test['CapGainsLossesPositive'], mar_cat_test_pd), axis=1)

X_adult_names = ['CapGainsLossesPositive', 'Divorced', 'Married', 'Never-married', 'Separated', 'Widowed']
Y_adult_names = ['<=50K', '>50K']

In [ ]:
#X_adult.head()
y_adult_test = adult_test['Income_factor']

In [ ]:
print('\n Test Accuracy: %.4f' % (model1.evaluate(X_adult_test, y_adult_test)[1]))

In [ ]:
y_adult_prediction = model1.predict(X_adult_test).flatten()
adult_pred = pd.concat((pd.DataFrame(y_adult_prediction), y_adult_test), axis=1) 

adult_tab = pd.crosstab(y_adult_test, y_adult_prediction, rownames=['Actual'], colnames=['Prediction'])
adult_tab

In [ ]:
### not working
y_prediction = model1.predict(X_adult_test).flatten()
for i in range(10):
    label = y_adult_test[i]
    prediction = y_adult_prediction[i]
print('actual value: {:.3f}, predicted value: {:.3f}'.format(label,prediction))

### 19 ???????????????????????
Plot NNM1 

In [ ]:
#pip install ann_visualizer

In [ ]:
from ann_visualizer.visualize import ann_viz

In [ ]:
ann_viz(model1, title='NNM1', view=True)

### 20
Evaluate NNM1 using Test set,Construct Contingency table to compare actual and predicted value

In [ ]:
adult_tab # All 0 on prediction??

### 21
Which baseline model do we compare NNM1 against?, NNM1 outperform baseline model based on accuracy?

In [ ]:
# ???

### 22
Gather Contingency table from CART model (CARTM1), C5.0 model (C5M1), Naive Bayes model (NBM1) from Ch6, Ch8


* CART (CARTM1)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.tools.tools as stattools
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

#folder = "/gdrive/My Drive/DSPR/DataSets"

adult_train = pd.read_csv(folder + "/adult_ch6_training", delimiter=',')
adult_test = pd.read_csv(folder + "/adult_ch6_test", delimiter=',')

In [ ]:
mar_np = np.array(adult_train['Marital status'])
(mar_cat, mar_cat_dict) = stattools.categorical(mar_np, drop=True, dictnames=True)

mar_cat_pd = pd.DataFrame(mar_cat)
X_adult_tree = pd.concat((adult_train['Cap_Gains_Losses'], mar_cat_pd), axis=1)

y_adult_tree = adult_train[['Income']]

X_adult_names_tree = ["Cap_Gains_Losses", "Divorced", "Married","Never-married","Separated", "Widowed"]
y_adult_names_tree = ["<=50K", ">50K"]

CARTM1 = DecisionTreeClassifier(criterion = "gini", max_leaf_nodes = 5).fit(X_adult_tree, y_adult_tree)
export_graphviz(CARTM1, out_file = folder+"/CARTM1.dot", feature_names = X_adult_names_tree, class_names = adult_train['Income'].unique())
### CART model using Training set 
Source.from_file(folder+"/CARTM1.dot")

In [ ]:
### Predict Test set

mar_np_test = np.array(adult_test['Marital status'])
(mar_cat_test, mar_cat_dict_test) = stattools.categorical(mar_np_test, drop=True, dictnames=True)

In [ ]:
mar_cat_test_pd = pd.DataFrame(mar_cat_test)
X_adult_tree_test =  pd.concat((adult_test['Cap_Gains_Losses'], mar_cat_test_pd), axis=1)

y_adult_tree_test = adult_test[['Income']]

X_adult_names_tree = ["Cap_Gains_Losses", "Divorced", "Married","Never-married","Separated", "Widowed"]
y_adult_names_tree = ["<=50K", ">50K"]

adult_CART_pred = CARTM1.predict(X_adult_tree_test)

adult_CART_crosstab = pd.crosstab(adult_test['Income'], adult_CART_pred)
adult_CART_crosstab

In [ ]:
def eval(tab, level):
    GT = sum(tab.sum())
    TAP = tab.sum(1)[1]
    TAN = tab.sum(1)[0]
    TPP = tab.sum(0)[1]
    TP = tab[level[1]][1]
    TN = tab[level[0]][0]

    acc = (TN + TP) / GT
    err = 1 - acc
    sen = TP / TAP
    spe = TN / TAN
    pre = TP / TPP
    
    return [round(acc, 4), round(err,4), round(sen,4), round(spe,4), round(pre,4)]

def Fscore(pre, spe, df):
    val = ((1+df**2) * pre * spe) / ((df**2 * pre) + spe)
    return round(val, 4)

In [ ]:
acc_cart, err_cart, sen_cart, spe_cart, pre_cart = eval(adult_CART_crosstab, y_adult_names_tree)
acc_cart, err_cart, sen_cart, spe_cart, pre_cart

In [ ]:
Fscore(pre_cart, spe_cart, 1), Fscore(pre_cart, spe_cart, 2), Fscore(pre_cart, spe_cart, 0.5)

In [ ]:
measure_cart_01 = pd.DataFrame(eval(adult_CART_crosstab, y_adult_names_tree))
measure_cart_02 = Fscore(pre_cart, spe_cart, 1), Fscore(pre_cart, spe_cart, 2), Fscore(pre_cart, spe_cart, 0.5)
measure_cart_02 = pd.DataFrame(measure_cart_02)
measure_cart_02

eval_cart = pd.concat((measure_cart_01, measure_cart_02), axis=0)
eval_cart.columns = ['CART']
eval_cart.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_cart

* C5.0 (C5M1)

In [ ]:
C5M1 = DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=5).fit(X_adult_tree, y_adult_tree)
export_graphviz(C5M1, out_file = folder+"/C5M1.dot", feature_names = X_adult_names_tree, class_names = adult_train['Income'].unique())
### C5.0 model using Training set 
Source.from_file(folder+"/C5M1.dot")

In [ ]:
### Predict Test set

adult_C5M1_pred = C5M1.predict(X_adult_tree_test)

adult_C5M1_crosstab = pd.crosstab(adult_test['Income'], adult_C5M1_pred)
adult_C5M1_crosstab

In [ ]:
acc_c5m1, err_c5m1, sen_c5m1, spe_c5m1, pre_c5m1 = eval(adult_C5M1_crosstab, y_adult_names_tree)
acc_c5m1, err_c5m1, sen_c5m1, spe_c5m1, pre_c5m1

In [ ]:
Fscore(pre_c5m1, spe_c5m1, 1), Fscore(pre_c5m1, spe_c5m1, 2), Fscore(pre_c5m1, spe_c5m1, 0.5)

In [ ]:
measure_cm51_01 = pd.DataFrame(eval(adult_C5M1_crosstab, y_adult_names_tree))
measure_cm51_02 = Fscore(pre_c5m1, spe_c5m1, 1), Fscore(pre_c5m1, spe_c5m1, 2), Fscore(pre_c5m1, spe_c5m1, 0.5)
measure_cm51_02 = pd.DataFrame(measure_cm51_02)
measure_cm51_02

eval_cm51 = pd.concat((measure_cm51_01, measure_cm51_02), axis=0)
eval_cm51.columns = ['C5.0']
eval_cm51.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_cm51

* Naive Bayes (NBM1)

In [ ]:
adult_train.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
import statsmodels.tools.tools as stattools

def X_ind_ftn(data, col):
    X_ind = np.array(data[col])
    X_ind = pd.DataFrame(stattools.categorical(X_ind,drop=True, dictnames = False))
    return X_ind

X_Mar_ind = X_ind_ftn(adult_train, "Marital status")
    
X_naive = pd.concat((X_Mar_ind, adult_train['Cap_Gains_Losses']), axis=1)
y_naive = adult_train["Income"]

nb_adult = MultinomialNB().fit(X_naive,y_naive)

In [ ]:
# Test NB algorithm by Test set
X_Mar_ind_test = X_ind_ftn(adult_test, 'Marital status')

X_naive_test = pd.concat((X_Mar_ind_test, adult_test['Cap_Gains_Losses']), axis=1)
y_naive_test = adult_test['Income']

adult_naive_pred = nb_adult.predict(X_naive_test)

In [ ]:
adult_naive_crosstab = pd.crosstab(y_naive_test, adult_naive_pred, rownames=['Actual'], colnames=['Predicted'])
adult_naive_crosstab

In [ ]:
acc_naive, err_naive, sen_naive, spe_naive, pre_naive = eval(adult_naive_crosstab, y_adult_names_tree)
acc_naive, err_naive, sen_naive, spe_naive, pre_naive

In [ ]:
Fscore(pre_naive, spe_naive, 1), Fscore(pre_naive, spe_naive, 2), Fscore(pre_naive, spe_naive, 0.5)

In [ ]:
measure_naive_01 = pd.DataFrame(eval(adult_naive_crosstab, y_adult_names_tree))
measure_naive_02 = Fscore(pre_naive, spe_naive, 1), Fscore(pre_naive, spe_naive, 2), Fscore(pre_naive, spe_naive, 0.5)
measure_naive_02 = pd.DataFrame(measure_naive_02)
measure_naive_02

eval_naive = pd.concat((measure_naive_01, measure_naive_02), axis=0)
eval_naive.columns = ['Naive Bayes']
eval_naive.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_naive

### 23
Compare Accuracy, Sensitivity, Specificity for each models (CART, C5.0, NBM1, NNM1)

In [ ]:
pd.concat((eval_cart, eval_cm51, eval_naive), axis=1)
# should have added Neural Network model....

### 24
Prepare dataset for Neural Networks including standardizing variables 

In [ ]:
bank_train = pd.read_csv(folder + "/bank_marketing_training", delimiter=',')
bank_test = pd.read_csv(folder + "/bank_marketing_test", delimiter=',')

In [ ]:
import statsmodels.tools.tools as stattools

# change 'Response' to numeric(binary)
bank_train["response_factor"] = list(map(int, bank_train["response"] == 'yes'))

# convert 'job', 'marital', 'education' to factor
# job
job_np = np.array(bank_train['job'])
(job_cat, job_cat_dict) = stattools.categorical(job_np, drop=True, dictnames=True)
# marital
mari_np = np.array(bank_train['marital'])
(mari_cat, mari_cat_dict) = stattools.categorical(mari_np, drop=True, dictnames=True)
# education
edu_np = np.array(bank_train['education'])
(edu_cat, edu_cat_dict) = stattools.categorical(edu_np, drop=True, dictnames=True)

In [ ]:
bank_train['job'].unique()

In [ ]:
job_cat_pd = pd.DataFrame(job_cat)
mari_cat_pd = pd.DataFrame(mari_cat)
edu_cat_pd = pd.DataFrame(edu_cat)

X_bank = pd.concat((bank_train['age'], job_cat_pd, mari_cat_pd, edu_cat_pd), axis=1)

#X_bank_names = [ ... ]
#Y_bank_names = ['yes', 'no']

In [ ]:
X_bank.head()

In [ ]:
#X_bank.head()
y_bank = bank_train['response_factor']

In [ ]:
model2 = Sequential()
model2.add(Dense(30, input_dim = 25, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model2.fit(X_bank, y_bank, epochs=5, batch_size=1)

In [ ]:
print('\n Accuracy: %.4f' % (model2.evaluate(X_bank, y_bank)[1]))

In [ ]:
bank_test["response_factor"] = list(map(int, bank_test["response"] == 'yes'))

job_np_test = np.array(bank_test['job'])
(job_cat_test, job_cat_dict_test) = stattools.categorical(job_np_test, drop=True, dictnames=True)

mari_np_test = np.array(bank_test['marital'])
(mari_cat_test, mari_cat_dict_test) = stattools.categorical(mari_np_test, drop=True, dictnames=True)

edu_np_test = np.array(bank_test['education'])
(edu_cat_test, edu_cat_dict_test) = stattools.categorical(edu_np_test, drop=True, dictnames=True)

In [ ]:
job_cat_test_pd = pd.DataFrame(job_cat_test)
mari_cat_test_pd = pd.DataFrame(mari_cat_test)
edu_cat_test_pd = pd.DataFrame(edu_cat_test)

X_bank_test = pd.concat((bank_test['age'], job_cat_test_pd, mari_cat_test_pd, edu_cat_test_pd), axis=1)

#X_bank_names = [ ... ]
#Y_bank_names = ['yes', 'no']

In [ ]:
#X_bank_test.head()
y_bank_test = bank_test['response_factor']

In [ ]:
print('\n Test Accuracy: %.4f' % (model2.evaluate(X_bank_test, y_bank_test)[1]))

In [ ]:
y_bank_prediction = model2.predict(X_bank_test).flatten()
bank_pred = pd.concat((pd.DataFrame(y_bank_prediction), y_bank_test), axis=1) 

bank_tab = pd.crosstab(y_bank_test, y_bank_prediction, rownames=['Actual'], colnames=['Prediction'])
bank_tab

In [ ]:
# ????????????????????????? all 1 on prediction? 

### 25
Using training set, create Neural Networks model, Obtain Predicted response

In [ ]:
y_bank_prediction # all 0 ????

### 26
Plot Neural Network

In [ ]:
# ????????????

### 27
Evaluate NN model using Test set, Construct contingency table to compare actual and predicted value

In [ ]:
bank_tab

### 28
Which baseline model do we compare NN model against?, Outperform baseline model based on accuracy? 

In [ ]:
# ???

### 29
Build CART, C5.0, Naive Bayes model using same predictors
* CART

In [ ]:
bank_train.head() 

In [ ]:
bank_train['education'].unique()

In [ ]:
def X_ind_ftn(data, col):
    X_ind = np.array(data[col])
    X_ind = pd.DataFrame(stattools.categorical(X_ind,drop=True, dictnames = False))
    return X_ind

X_job_ind = X_ind_ftn(bank_train, "job")
X_marital_ind = X_ind_ftn(bank_train, "marital")
X_edu_ind = X_ind_ftn(bank_train, "education")

X_bank_tree = pd.concat((bank_train[['age']],X_job_ind, X_marital_ind, X_edu_ind), axis=1)
X_bank_tree

y_bank_tree = bank_train[['response']]

X_bank_names_tree = ["age", "housemaid", "services","blue-collar","management", "unemployed", "retired", "technician", "admin.","unknown","entrepreneur","student","self-employed",
                     "married","single","divorced","unknown",
                     "basic.4y","high.school","unknown","basic.6y","basic.9y","university.degree","prefessional.course","illiterate"]
y_bank_names_tree = ["yes", "no"]

In [ ]:
CART_bank = DecisionTreeClassifier(criterion = "gini", max_leaf_nodes = 5).fit(X_bank_tree, y_bank_tree)
export_graphviz(CART_bank, out_file = folder+"/CART_bank.dot", feature_names = X_bank_names_tree, class_names = bank_train['response'].unique())
### CART model using Training set 
Source.from_file(folder+"/CART_bank.dot")

In [ ]:
X_job_ind_test = X_ind_ftn(bank_test, "job")
X_marital_ind_test = X_ind_ftn(bank_test, "marital")
X_edu_ind_test = X_ind_ftn(bank_test, "education")

X_bank_tree_test =  pd.concat((bank_test['age'], X_job_ind_test, X_marital_ind_test, X_edu_ind_test), axis=1)

y_bank_tree_test = bank_test[['response']]

bank_CART_pred = CART_bank.predict(X_bank_tree_test)

bank_CART_crosstab = pd.crosstab(bank_test['response'], bank_CART_pred, rownames=["Actual"], colnames=["Predicted"])
bank_CART_crosstab  # no yes??

In [ ]:
acc_bank_cart, err_bank_cart, sen_bank_cart, spe_bank_cart, pre_bank_cart = eval(bank_CART_crosstab, y_bank_names_tree)
acc_bank_cart, err_bank_cart, sen_bank_cart, spe_bank_cart, pre_bank_cart

Fscore(pre_bank_cart, spe_bank_cart, 1), Fscore(pre_bank_cart, spe_bank_cart, 2), Fscore(pre_bank_cart, spe_bank_cart, 0.5)

In [ ]:
bank_cart_01 = pd.DataFrame(eval(bank_CART_crosstab, y_bank_names_tree))
bank_cart_02 = Fscore(pre_bank_cart, spe_bank_cart, 1), Fscore(pre_bank_cart, spe_bank_cart, 2), Fscore(pre_bank_cart, spe_bank_cart, 0.5)
bank_cart_02 = pd.DataFrame(bank_cart_02)
bank_cart_02

In [ ]:
eval_bank_cart = pd.concat((bank_cart_01, bank_cart_02), axis=0)
eval_bank_cart.columns = ['CART']
eval_bank_cart.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_bank_cart

* C5.0

In [ ]:
C5M1_bank = DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=5).fit(X_bank_tree, y_bank_tree)
export_graphviz(C5M1_bank, out_file = folder+"/C5M1_bank.dot", feature_names = X_bank_names_tree, class_names = bank_train['response'].unique())
### C5.0 model using Training set 
Source.from_file(folder+"/C5M1_bank.dot")

In [ ]:
### Predict Test set

bank_C5M1_pred = C5M1_bank.predict(X_bank_tree_test)

bank_C5M1_crosstab = pd.crosstab(bank_test['response'], bank_C5M1_pred)
bank_C5M1_crosstab # no yes??

In [ ]:
acc_bank_c5m1, err_bank_c5m1, sen_bank_c5m1, spe_bank_c5m1, pre_bank_c5m1 = eval(bank_C5M1_crosstab, y_bank_names_tree)
acc_bank_c5m1, err_bank_c5m1, sen_bank_c5m1, spe_bank_c5m1, pre_bank_c5m1

In [ ]:
Fscore(pre_bank_c5m1, spe_bank_c5m1, 1), Fscore(pre_bank_c5m1, spe_bank_c5m1, 2), Fscore(pre_bank_c5m1, spe_bank_c5m1, 0.5)

In [ ]:
bank_cm51_01 = pd.DataFrame(eval(bank_C5M1_crosstab, y_bank_names_tree))
bank_cm51_02 = Fscore(pre_bank_c5m1, spe_bank_c5m1, 1), Fscore(pre_bank_c5m1, spe_bank_c5m1, 2), Fscore(pre_bank_c5m1, spe_bank_c5m1, 0.5)
bank_cm51_02 = pd.DataFrame(bank_cm51_02)
bank_cm51_02

eval_bank_cm51 = pd.concat((bank_cm51_01, bank_cm51_02), axis=0)
eval_bank_cm51.columns = ['C5.0']
eval_bank_cm51.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_bank_cm51

* Naive Bayes

In [ ]:
####

### 30
Compare Accuracy, Sensitivity, Specificity for above models

In [ ]:
####

### **CH 10 CLUSTERING**

### 10.5 K-MEANS CLUSTERING

In [ ]:
import pandas as pd
import numpy as np 
from scipy import stats
from sklearn.cluster import KMeans

In [ ]:
wine_train = pd.read_csv(folder + "/white_wine_training", delimiter=',')
wine_test = pd.read_csv(folder + "/white_wine_test", delimiter=',')

X = wine_train[['alcohol', 'sugar']]
Xz = pd.DataFrame(stats.zscore(X), columns=['alcohol', 'sugar'])

X_test = wine_test[['alcohol', 'sugar']]
Xz_test = pd.DataFrame(stats.zscore(X_test), columns=['alcohol', 'sugar'])

In [ ]:
kmeans01 = KMeans(n_clusters=2).fit(Xz)
cluster = kmeans01.labels_

In [ ]:
Cluster1 = Xz.loc[cluster == 0]
Cluster2 = Xz.loc[cluster == 1]

In [ ]:
Cluster1.describe(), Cluster2.describe()

In [ ]:
kmeans_test = KMeans(n_clusters=2).fit(Xz_test)
cluster_test = kmeans_test.labels_ # Cluster membership
Cluster1_test = Xz_test.loc[cluster_test == 0]
Cluster2_test = Xz_test.loc[cluster_test == 1]

Cluster1_test.describe(), Cluster2_test.describe()

### **HANDS-ON ANALYSIS**


### 15
Subset 'Fat' and 'Sodium' variables, Standardize dataset

In [ ]:
cereals = pd.read_csv(folder + "/cereals.CSV", delimiter=',')

X_cereals = cereals[['Fat', 'Sodium']]
X_cereals_z = pd.DataFrame(stats.zscore(X_cereals), columns=['Fat', 'Sodium'])

X_cereals_z.head()

### 16
Run K-means clustering using 3 clusters

In [ ]:
kmeans_cereals = KMeans(n_clusters=3).fit(X_cereals_z)
cluster_cereals = kmeans_cereals.labels_

In [ ]:
clust1_c = X_cereals_z.loc[cluster_cereals == 0]
clust2_c = X_cereals_z.loc[cluster_cereals == 1]
clust3_c = X_cereals_z.loc[cluster_cereals == 2]

In [ ]:
clust1_c.describe(), clust2_c.describe(), clust3_c.describe()

### 17
Obtain summary of each variable within each cluster, and use summary to identify below
* a. A low Fat, low Sodium cluster


In [ ]:
#clust1_c.describe()
#clust2_c.describe()
#clust3_c.describe()

In [ ]:
# ?????????????????????????????????????????????????????????

* b. A low Fat, high Sodium cluster

In [ ]:
# ?????????????????????????????????????????????????????????

* c. A high Fat, high Sodium cluster

In [ ]:
# ?????????????????????????????????????????????????????????

### 18
Run K-means clustering with k=2

In [ ]:
fram_train = pd.read_csv(folder + "/Framingham_Training", delimiter=',')
fram_test = pd.read_csv(folder + "/Framingham_Test", delimiter=',')

X_fram = fram_train[['Sex', 'Age']]

Age_z = pd.DataFrame(stats.zscore(X_fram['Age']), columns=['Age'])
X_fram_z = pd.concat((X_fram['Sex'], Age_z), axis=1)

X_fram_z.head()

In [ ]:
kmeans_fram = KMeans(n_clusters=2).fit(X_fram_z)
cluster_fram = kmeans_fram.labels_

### 19
Construct a table of statistics summarizing clusters, Describe what these clusters consist of

In [ ]:
clust1_f = X_fram_z.loc[cluster_fram == 0]
clust2_f = X_fram_z.loc[cluster_fram == 1]

In [ ]:
clust1_f.describe(), clust2_f.describe()

In [ ]:
# Describe above stat

### 20 
Perform K-means clustering on Test set with K=2 clusters


In [ ]:
X_fram_test = fram_test[['Sex', 'Age']]

Age_z_test = pd.DataFrame(stats.zscore(X_fram_test['Age']), columns=['Age'])
X_fram_z_test = pd.concat((X_fram_test['Sex'], Age_z_test), axis=1)

X_fram_z_test.head()

In [ ]:
kmeans_fram_test = KMeans(n_clusters=2).fit(X_fram_z_test)
cluster_fram_test = kmeans_fram_test.labels_ # Cluster membership
Cluster1_fram_test = X_fram_z_test.loc[cluster_fram_test == 0]
Cluster2_fram_test = X_fram_z_test.loc[cluster_fram_test == 1]

Cluster1_fram_test.describe(), Cluster2_fram_test.describe()

### 21
Report results from test set, Clusters are validated?


In [ ]:
# describe

### 22
Run K-means on Training set with K=3 clusters

In [ ]:
kmeans_fram_3clust = KMeans(n_clusters=3).fit(X_fram_z)
cluster_fram_3clust = kmeans_fram_3clust.labels_

### 23 
Construct table of stat summarizing clusters 

In [ ]:
clust1_f_3clust = X_fram_z.loc[cluster_fram_3clust == 0]
clust2_f_3clust = X_fram_z.loc[cluster_fram_3clust == 1]
clust3_f_3clust = X_fram_z.loc[cluster_fram_3clust == 2]

In [ ]:
clust1_f_3clust.describe(), clust2_f_3clust.describe(), clust3_f_3clust.describe()

In [ ]:
# Describe

### 24
Perform K-means clustering on Test set with K=3 clusters

In [ ]:
X_fram_test = fram_test[['Sex', 'Age']]

Age_z_test = pd.DataFrame(stats.zscore(X_fram_test['Age']), columns=['Age'])
X_fram_z_test = pd.concat((X_fram_test['Sex'], Age_z_test), axis=1)

In [ ]:
kmeans_fram_3clust_test = KMeans(n_clusters=3).fit(X_fram_z_test)
cluster_fram_3clust_test = kmeans_fram_3clust_test.labels_ # Cluster membership
Cluster1_fram_3clust_test = X_fram_z_test.loc[cluster_fram_3clust_test == 0]
Cluster2_fram_3clust_test = X_fram_z_test.loc[cluster_fram_3clust_test == 1]
Cluster3_fram_3clust_test = X_fram_z_test.loc[cluster_fram_3clust_test == 2]

Cluster1_fram_3clust_test.describe(), Cluster2_fram_3clust_test.describe(), Cluster3_fram_3clust_test.describe()

In [ ]:
# Describe


### 25
Report result from test set, are cluaters validated?

In [ ]:
# Describe

### 26
Run K-means clustering with k=4

In [ ]:
kmeans_fram_4clust = KMeans(n_clusters=4).fit(X_fram_z)
cluster_fram_4clust = kmeans_fram_4clust.labels_

### 27 
Construct a table of stat summarizing 4 clusters

In [ ]:
clust1_f_4clust = X_fram_z.loc[cluster_fram_4clust == 0]
clust2_f_4clust = X_fram_z.loc[cluster_fram_4clust == 1]
clust3_f_4clust = X_fram_z.loc[cluster_fram_4clust == 2]
clust4_f_4clust = X_fram_z.loc[cluster_fram_4clust == 3]

In [ ]:
clust1_f_4clust.describe(), clust2_f_4clust.describe(), clust3_f_4clust.describe(), clust4_f_4clust.describe()

In [ ]:
# Describe

### 28
Perform K-means clustering on Test set with K=4 clusters

In [ ]:
X_fram_test = fram_test[['Sex', 'Age']]

Age_z_test = pd.DataFrame(stats.zscore(X_fram_test['Age']), columns=['Age'])
X_fram_z_test = pd.concat((X_fram_test['Sex'], Age_z_test), axis=1)

In [ ]:
kmeans_fram_4clust_test = KMeans(n_clusters=4).fit(X_fram_z_test)
cluster_fram_4clust_test = kmeans_fram_4clust_test.labels_ # Cluster membership
Cluster1_fram_4clust_test = X_fram_z_test.loc[cluster_fram_4clust_test == 0]
Cluster2_fram_4clust_test = X_fram_z_test.loc[cluster_fram_4clust_test == 1]
Cluster3_fram_4clust_test = X_fram_z_test.loc[cluster_fram_4clust_test == 2]
Cluster4_fram_4clust_test = X_fram_z_test.loc[cluster_fram_4clust_test == 3]

Cluster1_fram_4clust_test.describe(), Cluster2_fram_4clust_test.describe(), Cluster3_fram_4clust_test.describe(), Cluster4_fram_4clust_test.describe()

In [ ]:
# Describe

### 29 
Report result from test set, are 4 clustesrs validated? 

In [ ]:
# Describe

### 30
Which clustering solution, k= 2,3,4 do you prefer? why?

In [ ]:
# Describe